In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import csv
from scipy import optimize

from scipy.signal import savgol_filter

from hyperopt import hp, tpe, Trials, fmin
import hyperopt

In [2]:
# helper functions

X_train = []
y_train = []


# SINE REGRESSION FUNCTIONS __________________________________________________________________________

def objective(a1, w, f, a0):
    return np.mean((a1 * np.sin(w*X_train + f) + a0 - y_train)**2)

def objective2(args):
    return objective(*args)

def get_hyperopt_params():

    space = [hp.uniform('a0', -1000, 0),
         hp.uniform('a1', 100, 800), 
         hp.uniform('w', 0.0, 0.5), 
         hp.uniform('f', -2 * np.pi, 0)]

    tpe_algo = tpe.suggest
    tpe_trials = Trials()

    tpe_best = fmin(fn=objective2, space=space, algo=tpe_algo, trials=tpe_trials, max_evals=300)

    amplitude = float('{a1:.2f}'.format(**tpe_best))
    constant = float('{a0:.2f}'.format(**tpe_best))
    horiz_scale = float('{w:.2f}'.format(**tpe_best))
    offset = float('{f:.2f}'.format(**tpe_best))

    losses = tpe_trials.losses()

    return [amplitude, horiz_scale, offset, constant, min(losses)]

def get_best_hyperopt_params():

    min_loss = -1
    best_amp = 0
    best_w = 0
    best_offset = 0
    best_const  = 0

    for i in range(20):
        ret = get_hyperopt_params()
        if (ret[4] < min_loss or min_loss < 0):
            best_amp, best_w, best_offset, best_const, min_loss = ret
    
    return [best_amp, best_w, best_offset, best_const]

def sinusoid(x, a, b, c, d):
    return a * np.sin(b * x + c) + d

def scipy_opt(hyperopt_params):
    params, params_covariance = optimize.curve_fit(sinusoid, X_train, y_train, p0=hyperopt_params, method='lm')
    return params

def regress_sine_wave():
    hp_params = get_best_hyperopt_params()
    final_params = scipy_opt(hp_params)
    return final_params
# ____________________________________________________________________________________________________________________

# MISC FUNCTIONS __________________________________________________________________________________________

def swap_lr(series):
    for l_key in series:
        if (l_key[0] == 'l'):
            r_key = 'right' + l_key[4:]
            temp = series[l_key]
            series[l_key] = series[r_key]
            series[r_key] = temp
    return series


def series_append(series, list, keys):
    for i in range(64):
        series[keys[i]].append(float(list[i]))
    return series

def load_series(filename):
    with open(filename, 'r') as csv_in:
        csv_file = list(csv.reader(csv_in))
        series = {}
        keys = csv_file[0]
        for key in keys: series[key] = []
        for i in range(2, len(csv_file), 2):
            series = series_append(series, csv_file[i], keys)
        return [series, int((len(csv_file) - 2) / 2)]

# _______________________________________________________________________________________________________________

In [5]:
# [1] Find correct sides, offsets, and periods

scale_normalized_dir = '..\\time_series\\scale_normalized\\'
scale_normalized_series = os.listdir(scale_normalized_dir)

offset_list = []
period_list = []
side_list = []

def get_sine_params(filename):
    
    series, num_frames = load_series(filename)
    
    X_train = np.arange(0, num_frames)

    y_train = savgol_filter(series['left_heel_y'], window_length=11, polyorder=3)
    l_amp, l_omega, l_phi, l_const = regress_sine_wave()

    y_train = savgol_filter(series['right_heel_y'], window_length=11, polyorder=3)
    r_amp, r_omega, r_phi, r_const = regress_sine_wave()

    l_period = (2 * np.pi) / float(l_omega)
    r_period = (2 * np.pi) / float(r_omega)
    
    l_offset = (0.5 * np.pi / float(l_omega)) - (float(l_phi) / float(l_omega))
    r_offset = (0.5 * np.pi / float(r_omega)) - (float(r_phi) / float(r_omega))

    if (l_amp < 0.0):
        l_offset -= (np.pi) / (float(l_omega))
    if (r_amp < 0.0):
        r_offset -= (np.pi) / float(r_omega)
    
    while (l_offset < 0.0): 
        l_offset += l_period
    while (r_offset < 0.0): 
        r_offset += r_period

    # PLOTTING _____________________________________________________________________________________________________

    fig, ax = plt.subplots(2, figsize=(20, 10))
    sin_ly = []
    sin_ry = []
    ly = savgol_filter(series['left_heel_y'], window_length=11, polyorder=3)
    ry = savgol_filter(series['right_heel_y'], window_length=11, polyorder=3)
    for i in X_train:
        sin_ly.append(sinusoid(i, l_amp, l_omega, l_phi, l_const))
        sin_ry.append(sinusoid(i, r_amp, r_omega, r_phi, r_const))
    ax[0].plot(X_train, ly)
    ax[1].plot(X_train, ry)
    ax[0].plot(X_train, sin_ly)
    ax[1].plot(X_train, sin_ry)
    ax[0].axvline(l_offset, color='r')
    ax[1].axvline(r_offset, color='r')

    # print('LEFT: ' + str(l_amp) + ' * sin( ' + str(l_omega) + ' * x + ' + str(l_phi) + ' ) + ' + str(l_const)) 
    # print('RIGHT: ' + str(r_amp) + ' * sin( ' + str(r_omega) + ' * x + ' + str(r_phi) + ' ) + ' + str(r_const)) 

    ax[0].axvline(l_offset + l_period, color='green')
    ax[1].axvline(r_offset + r_period, color='green')

    # ________________________________________________________________________________________________________________

    return [l_amp, l_omega, l_phi, l_const, r_amp, r_omega, r_phi, r_const]


for file in scale_normalized_series:
    filename = scale_normalized_dir + file

    sine_params = get_sine_params(filename)




    break



In [4]:
''' LAYOUT OF TEMPORAL OPTIMIZATION PIPELINE

define lists for offset and period
define objective functions for left and right

[1] find correct sides, offsets, and periods

for each scale normalized series:

    load the series

    regress left and right heel_y's:
        5 iterations of hyperopt, store params of the one with lowest best loss
        run scipy with hyperopt lowest loss as init guess

    save period as 2pi / omega
    save offset as (0.5 * pi / omega) - (phi/b)
        add period until offset is greater than 0
    choose side with lower offset
    
    if b, try again
    if s, skip it and make a note of the number skipped for removal

[2] trim examples

find minimum period

for each scale normalized file:
    if foot and entryside mismatch:
        swap all L-R coords (helper function)

    find number of examples: min (((total frames - offset) / period), 3)
    find step size: current_period / min_period

    define a new series and step through at the step size, picking values to put in the normalized list

    for each number of examples:
        define a new series and append all the stepped frame values

'''

" LAYOUT OF TEMPORAL OPTIMIZATION PIPELINE\n\ndefine lists for offset and period\ndefine objective functions for left and right\n\n[1] find correct sides, offsets, and periods\n\nfor each scale normalized series:\n\n    load the series\n\n    regress left and right heel_y's:\n        5 iterations of hyperopt, store params of the one with lowest best loss\n        run scipy with hyperopt lowest loss as init guess\n\n    save period as 2pi / omega\n    save offset as (0.5 * pi / omega) - (phi/b)\n        add period until offset is greater than 0\n    choose side with lower offset\n    \n    if b, try again\n    if s, skip it and make a note of the number skipped for removal\n\n[2] trim examples\n\nfind minimum period\n\nfor each scale normalized file:\n    if foot and entryside mismatch:\n        swap all L-R coords (helper function)\n\n    find number of examples: min (((total frames - offset) / period), 3)\n    find step size: current_period / min_period\n\n    define a new series and 